# 01. Overview

#### 가설
* 가설1 : 거주 인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설2 : 직장이 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설3 : 유동인구가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설4 : 역 근처에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설5 : 땅값이 비싼지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설6 : 대학교 근처에 스타벅스 매장이 많이 입지해 있을것이다.
* 가설7 : 인근 카페가 많을 수록 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설8 : 거주 인구가 20~30대가 많은 지역에 스타벅스 매장이 많이 입지해 있을 것이다.
* 가설9 : 교통편이 편리한(지하철, 버스 등) 지역에 스타벅스 매장이 많을 입지해 있을 것이다.

* 스타벅스 입지전략을 분석 : 가설1,2에 대한 검증 진행

#### 데이터 수집
* 1. 스타벅스 사이트에서 서울시에 존재하는 매장의 정보 수집
    * 매장명, 위도, 경도, 매장타입, 주소, 전화번호
    * https://www.starbucks.co.kr/

* 2. 서울시 주민등록 인구(구별)통계 정보 수집
    * 서울시 열린 데이터 광장
        * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 3. 서울시 사업체 현황(산업대분류별/동별) 통계 정보
    * 서울시 열린 데이터 광장
        * https://data.seoul.go.kr/dataList/104/S/2/datasetView.do

## 02.데이터 수집

### 2-1.서울시 스타벅스 매장 목록 데이터 수집:크롤링
* www.istarbucks.co.kr

In [1]:
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)x``

url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)
driver.implicitly_wait(10)
seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'

# WebDricerWait는 driver를 20초동안 대기시킴
# 조건 element_to_be_clickable : 클릭가능한 요소(seoul_btn)가 나타날때까지에 대한 조건
# 조건이 나타나면 seoul_btn를 클릭
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,seoul_btn))).click()
except TimeoutError:
    print('해당 페이지가 20초내에 열리지 않았음')

In [2]:
# '전체' 버튼 요소를 찾아 클릭
all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
try:
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,all_btn))).click()
except TimeoutError:
    print('해당 페이지가 20초내에 열리지 않았음')

In [3]:
# BeautifulSoup으로 HTML 파서 만들기
html = driver.page_source
soup = bs(html,'html.parser')

In [ ]:
#교수님버전
starbucks_soup_list = soup.select('li.quickResultLstCon')
print(len(starbucks_soup_list))
#강희 버전
st_li= soup.find('div',{'id':'mCSB_3_container'})
st_li.find_all('li',{'class':'quickResultLstCon'})#[0].text.strip().split('   ')

In [5]:
name = starbucks_soup_list[0].find('strong').text.strip() #건물명
lat = starbucks_soup_list[0]['data-lat'].strip() # 위도
lng = starbucks_soup_list[0]['data-long'].strip() # 경도
store_type = starbucks_soup_list[0].select('i')[0]['class'][0] #매장타입
add_tel = str(starbucks_soup_list[0].select('p.result_details')[0]).split('<br/>')
address = add_tel[0].split('>')[1]
tel = add_tel[1].split('<')[0]
print(name)
print(lat)
print(lng)
print(store_type)
print(address)
print(tel)

역삼아레나빌딩
37.501087
127.043069
pin_general
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [6]:
name2 = []
lat2 = []
lng2 = []
store_type2 = []
add_tel2 = []
address2 =[]
tel2 = []
for i in starbucks_soup_list:
    name2.append(i.find('strong').text.strip())
    lat2.append(i['data-lat'].strip())
    lng2.append(i['data-long'].strip())
    store_type2.append(i.select('i')[0]['class'][0])
    add_tel2 = str(i.select('p.result_details')[0]).split('<br/>')
    address2.append(add_tel2[0].split('>')[1])
    tel2.append(add_tel[1].split('<')[0])


In [13]:
seoul_starbucks_df = pd.DataFrame({
    '매장명':name2,
    '위도':lat2,
    '경도':lng2,
    '매장타입':store_type2,
    '주소':address2,
    '전화번호':tel2
})

In [14]:
seoul_starbucks_df.to_csv('스타벅스_info.csv')

In [17]:
seoul_starbucks_df.to_excel('스타벅스_info.xlsx',index=False)

In [15]:
seoul_starbucks_df.head()

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,pin_general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,pin_general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,pin_general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,pin_general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,pin_general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


### 2-2. 서울시 통계 정보 수집 : 공공데이터
* 서울시 주민등록 인구(구별)통계 정보 수집 : Txt
    * https://data.seoul.go.kr/dataList/419/S/2/datasetView.do
* 서울시 사업체현황(산업대분류별/동별) 통계 정보 수집 : Txt
    * https://data.seoul.go.kr/dataList/104/S/2/datasetView.do

#### 2-2-1. 서울시 주민등록 인구(구별) 데이터 분석

In [18]:
sgg_pop_df = pd.read_csv('./files/report.txt',sep='\t',header=1)
sgg_pop_df

,시점,동별(1),동별(2),소계,소계.1,남자,여자,소계.2,남자.1,여자.1,소계.3,남자.2,여자.2,소계.4,소계.5
0,2023 1/4,합계,소계,4463385,9668008,4672170,4995838,9426404,4566299,4860105,241604,105871,135733,2.11,1690961
1,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
2,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
3,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
4,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
5,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854
6,2023 1/4,합계,동대문구,171140,354884,172178,182706,337574,165933,171641,17310,6245,11065,1.97,65154
7,2023 1/4,합계,중랑구,187898,389928,191122,198806,385003,189186,195817,4925,1936,2989,2.05,76116
8,2023 1/4,합계,성북구,197859,441855,210937,230918,430100,206634,223466,11755,4303,7452,2.17,78427
9,2023 1/4,합계,강북구,144557,296934,143493,153441,292762,142028,150734,4172,1465,2707,2.03,67056


In [19]:
# 서울시 시군구 인구 데이터 살펴보기
columns = {
    '시점': 'GIGAN',
    '동별(1)': 'SUM',
    '동별(2)': 'JACHIGU',
    '소계': 'SEDAE',
    '소계.1': 'GYE_1',
    '남자': 'NAMJA_1',
    '여자': 'YEOJA_1',
    '소계.2': 'GYE_2', # 한국인
    '남자.1': 'NAMJA_2',
    '여자.1': 'YEOJA_2',
    '소계.3': 'GYE_3', # 등록외국인
    '남자.2': 'NAMJA_3',
    '여자.2': 'YEOJA_3',
    '소계.4': 'SEDAEDANGINGU',
    '소계.5': 'N_65SEISANGGORYEONGJA'
}
sgg_pop_df.rename(columns = columns, inplace = True)
sgg_pop_df.head()

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2023 1/4,합계,소계,4463385,9668008,4672170,4995838,9426404,4566299,4860105,241604,105871,135733,2.11,1690961
1,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
2,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
3,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
4,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238


In [20]:
sgg_pop_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  26 non-null     object 
 1   SUM                    26 non-null     object 
 2   JACHIGU                26 non-null     object 
 3   SEDAE                  26 non-null     int64  
 4   GYE_1                  26 non-null     int64  
 5   NAMJA_1                26 non-null     int64  
 6   YEOJA_1                26 non-null     int64  
 7   GYE_2                  26 non-null     int64  
 8   NAMJA_2                26 non-null     int64  
 9   YEOJA_2                26 non-null     int64  
 10  GYE_3                  26 non-null     int64  
 11  NAMJA_3                26 non-null     int64  
 12  YEOJA_3                26 non-null     int64  
 13  SEDAEDANGINGU          26 non-null     float64
 14  N_65SEISANGGORYEONGJA  26 non-null     int64  
dtypes: float

In [22]:
sgg_pop_df.head()

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
0,2023 1/4,합계,소계,4463385,9668008,4672170,4995838,9426404,4566299,4860105,241604,105871,135733,2.11,1690961
1,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
2,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
3,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
4,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238


In [23]:
# 필요없는 데이터 제거(임시 참고)
condition = sgg_pop_df['JACHIGU'] != '소계'
sgg_pop_df_selected = sgg_pop_df[condition]
sgg_pop_df_selected.head()

,GIGAN,SUM,JACHIGU,SEDAE,GYE_1,NAMJA_1,YEOJA_1,GYE_2,NAMJA_2,YEOJA_2,GYE_3,NAMJA_3,YEOJA_3,SEDAEDANGINGU,N_65SEISANGGORYEONGJA
1,2023 1/4,합계,종로구,72679,152212,72819,79393,141060,68170,72890,11152,4649,6503,1.94,28265
2,2023 1/4,합계,중구,63862,131390,63576,67814,120963,58699,62264,10427,4877,5550,1.89,25353
3,2023 1/4,합계,용산구,109735,232482,112608,119874,217756,104640,113116,14726,7968,6758,1.98,39478
4,2023 1/4,합계,성동구,133513,287240,139171,148069,280240,136233,144007,7000,2938,4062,2.10,48238
5,2023 1/4,합계,광진구,169787,350925,167983,182942,336801,162209,174592,14124,5774,8350,1.98,54854


In [24]:
sgg_pop_df_selected.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GIGAN                  25 non-null     object 
 1   SUM                    25 non-null     object 
 2   JACHIGU                25 non-null     object 
 3   SEDAE                  25 non-null     int64  
 4   GYE_1                  25 non-null     int64  
 5   NAMJA_1                25 non-null     int64  
 6   YEOJA_1                25 non-null     int64  
 7   GYE_2                  25 non-null     int64  
 8   NAMJA_2                25 non-null     int64  
 9   YEOJA_2                25 non-null     int64  
 10  GYE_3                  25 non-null     int64  
 11  NAMJA_3                25 non-null     int64  
 12  YEOJA_3                25 non-null     int64  
 13  SEDAEDANGINGU          25 non-null     float64
 14  N_65SEISANGGORYEONGJA  25 non-null     int64  
dtypes: float

In [25]:
# 분석에 필요한 컬럼 선택
columns = ['JACHIGU','GYE_1']
sgg_pop_df_final = sgg_pop_df_selected[columns]
sgg_pop_df_final.head()

,JACHIGU,GYE_1
1,종로구,152212
2,중구,131390
3,용산구,232482
4,성동구,287240
5,광진구,350925


In [26]:
# 데이터프레임의 칼럼명 변경
sgg_pop_df_final.columns = ['시군구명','주민등록인구']
sgg_pop_df_final.info()
sgg_pop_df_final.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 1 to 25
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   시군구명    25 non-null     object
 1   주민등록인구  25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 600.0+ bytes


,시군구명,주민등록인구
1,종로구,152212
2,중구,131390
3,용산구,232482
4,성동구,287240
5,광진구,350925


In [27]:
# 엑셀로 저장
sgg_pop_df_final.to_excel('./files/sgg_pop.xlsx',index =False)

### 2-2-2. 서울시 사업체현황(산업대분류별/동별) 데이터 분석

In [38]:
sgg_biz_df = pd.read_csv('./files/report1.txt', sep='\t', header = 3)


In [39]:
columns = {
    '시점': 'GIGAN',
    '동별(1)': 'GIGAN',
    '동별(2)': 'JACHIGU',
    '동별(3)': 'DONG',
    '소계': 'SAEOPCHESU_1',
    '여성대표자': 'YEOSEONGDAEPYOJA',
    '소계.1': 'GYE',
    '남': 'NAM',
    '여': 'YEO'
}
sgg_biz_df.rename(columns = columns, inplace = True)
sgg_biz_df.head(10)


,GIGAN,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
1,2021,합계,종로구,사직동,5035,1708,56013,28597,27416,1,...,28,7047,132,927,120,1207,114,1612,285,944
2,2021,합계,종로구,삼청동,956,423,4978,2482,2496,-,...,3,1264,20,287,9,40,37,307,39,146
3,2021,합계,종로구,부암동,894,416,3762,1665,2097,-,...,3,65,55,1338,30,286,39,99,81,226
4,2021,합계,종로구,평창동,1321,577,4321,2068,2253,-,...,3,115,70,448,40,521,81,195,123,281
5,2021,합계,종로구,무악동,761,176,1891,960,931,-,...,1,14,33,168,10,560,14,27,51,168
6,2021,합계,종로구,교남동,592,256,8371,4445,3926,-,...,2,37,32,191,30,3568,17,64,57,154
7,2021,합계,종로구,가회동,969,436,14180,10707,3473,-,...,3,528,39,726,19,88,47,132,69,178
8,2021,합계,종로구,종로1.2.3.4가동,17286,5412,107067,63437,43630,1,...,24,4654,339,2890,221,1850,243,1031,911,2509
9,2021,합계,종로구,종로5.6가동,7972,2614,22872,14187,8685,1,...,5,352,35,154,94,441,50,136,252,764


In [42]:
sgg_biz_df[['DONG','SAEOPCHESU_1']]

,DONG,SAEOPCHESU_1
0,소계,48361
1,사직동,5035
2,삼청동,956
3,부암동,894
4,평창동,1321
...,...,...
446,천호2동,4863
447,길동,5966
448,강일동,1443
449,상일1동,1679


In [43]:
# 시군구별 사업체 현황 데이터 추출(행)
condition = sgg_biz_df['DONG']=='소계'
sgg_biz_df_selected = sgg_biz_df[condition]
sgg_biz_df_selected.head()

,GIGAN,GIGAN,JACHIGU,DONG,SAEOPCHESU_1,YEOSEONGDAEPYOJA,GYE,NAM,YEO,소계.2,...,소계.30,소계.31,소계.32,소계.33,소계.34,소계.35,소계.36,소계.37,소계.38,소계.39
0,2021,합계,종로구,소계,48361,16563,275063,153210,121853,4,...,100,17436,1082,13298,830,19143,1141,5650,2591,7297
18,2021,합계,중구,소계,70308,25794,386564,213782,172782,1,...,65,16141,644,9425,813,11585,632,3258,2175,7632
34,2021,합계,용산구,소계,29680,10454,152605,84314,68291,4,...,45,3416,844,7352,632,8219,651,3006,1866,5572
51,2021,합계,성동구,소계,41665,13447,203221,116978,86243,7,...,45,5243,1283,10453,815,9635,844,2956,2341,7334
69,2021,합계,광진구,소계,33706,12297,129707,71124,58583,3,...,38,3592,1467,14698,981,12785,1139,3202,2799,5121


In [44]:
# 필요없는 칼럼 제거(열)
columns = ['JACHIGU','GYE','SAEOPCHESU_1']
sgg_biz_df_final = sgg_biz_df_selected[columns]
sgg_biz_df_final.columns = ['시군구명','종사자수','사업체수']
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,275063,48361
18,중구,386564,70308
34,용산구,152605,29680
51,성동구,203221,41665
69,광진구,129707,33706


In [45]:
# 데이터프레임의 인덱스 초기화
sgg_biz_df_final = sgg_biz_df_final.reset_index(drop=True)
sgg_biz_df_final.head()

,시군구명,종사자수,사업체수
0,종로구,275063,48361
1,중구,386564,70308
2,용산구,152605,29680
3,성동구,203221,41665
4,광진구,129707,33706


In [46]:
sgg_biz_df_final.to_excel('./files/sgg_biz.xlsx',index=False)

In [47]:
'''
[file lists]
1. Data collection
- seoul_starbucks.xlsx : 스타벅스 정보(크롤링)
- seoul_sgg_list.xlsx  : 서울시 시군구 정보(공공데이터)
- report.txt           : 시군구별 인구수 현황(공공데이터)
- report1.txt          : 시군구별 사업체 현황(공공데이터)

2. Data processing
- sgg_pop.xlsx         : 시군구별 인구수 현황(가공)
- sgg_biz.xlsx         : 시군구별 사업체 현황(가공)
'''



'\n[file lists]\n1. Data collection\n- seoul_starbucks.xlsx : 스타벅스 정보(크롤링)\n- seoul_sgg_list.xlsx  : 서울시 시군구 정보(공공데이터)\n- report.txt           : 시군구별 인구수 현황(공공데이터)\n- report1.txt          : 시군구별 사업체 현황(공공데이터)\n\n2. Data processing\n- sgg_pop.xlsx         : 시군구별 인구수 현황(가공)\n- sgg_biz.xlsx         : 시군구별 사업체 현황(가공)\n'